<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/2022/ex12notebookC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2022 ex12notebookC

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo12.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2022

----
## 演習問題 - 猫顔画像のクラスタリング
----

猫顔画像131枚をクラスタリングしてみましょう．

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# SciPy の階層型クラスタリングパッケージ
import scipy.cluster.hierarchy as hierarchy

# scikit-learn の K-平均法のクラス
from sklearn.cluster import KMeans

---
### 準備

画像を並べて表示する関数を定義しておきます．

In [ ]:
#####  データの最初の nx x ny 枚を可視化
#
def mosaicImage(ax, dat, nx, ny, nrow=64, ncol=64, gap=4):

    # 並べた画像の幅と高さ
    width  = nx * (ncol + gap) + gap
    height = ny * (nrow + gap) + gap

    # 画像の作成
    img = np.zeros((height, width), dtype=int) + 128
    for iy in range(ny):
        lty = iy*(nrow + gap) + gap
        for ix in range(nx):
            if iy*nx+ix >= dat.shape[0]:
                break
            ltx = ix*(ncol + gap) + gap
            img[lty:lty+nrow, ltx:ltx+ncol] = dat[iy*nx+ix, :].reshape((nrow, ncol))

    # 表示
    ax.axis('off')
    ax.imshow(img, cmap='gray')


猫顔画像データを入手して一部を表示してみます．

In [ ]:
# 猫顔画像データを入手
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/cat131.npz
cat = np.load('cat131.npz')['cat131']
N, D = cat.shape
print(f'データ数 x 次元数 = {N} x {D}')

# 最初の 20 枚を表示
fig, ax = plt.subplots(figsize=(6, 6))
mosaicImage(ax, cat, 5, 4)
plt.show()  

---
### 階層型クラスタリングしてみよう



上記の配列 `cat` に格納されたデータを対象に階層型クラスタリングを実行します．
[scipy.cluster.hiearchy.linkage](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html) でクラスタリングして，
[scipy.cluster.hierarchy.dendrogram](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html) でデンドログラムを描きます．
標本間の距離はユークリッド距離，クラスタ間の距離はウォード法で測ります．

In [ ]:
link = hierarchy.linkage(cat, method='ward', metric='euclidean')
fig, ax = plt.subplots(figsize=(8, 20))
hierarchy.dendrogram(link, orientation='right', distance_sort='descending', show_leaf_counts=True, leaf_font_size=8, ax=ax)
plt.show()

次のセルを実行すると，`threshold` で指定した値をクラスタ間距離のしきい値として画像をクラスタに分けて，それぞれのクラスタに属する画像の数を表示します．

In [ ]:
threshold = 20000 # クラスタ間距離のしきい値

label = hierarchy.fcluster(link, t=threshold, criterion='distance')
print(label)
K = max(label) # クラスタ数

print()
print(f'threshold = {threshold}, K = {K}')
for ik in range(K):
    print(f'cluster{ik+1}: num = {np.sum(label == ik+1)}')

次のセルを実行すると，上で得られたクラスタリングの結果を画像として表示します．一つのクラスタに属する画像を横に並べてあります．

In [ ]:
# 各クラスタに属する画像を表示

nc = 10 # 1行に並べる画像の最大数

dat = np.zeros((K, nc, D))
for ik in range(K):
    idx = label == (ik+1)
    n = np.sum(idx)
    if n < nc:
        dat[ik, :n, :] = cat[idx, :]
    else:
        dat[ik, :, :] = cat[idx, :][:nc, :]

dat = dat.reshape((-1, D))

fig, ax = plt.subplots(figsize=(12, 12))
mosaicImage(ax, dat, nc, K)
plt.show()  

### 問題1

次のことを考えて／調べてメモしておこう：

(1) この実験では何をやっている？どんな手法を使っている？

(2) 使っているデータのデータ数と次元数はいくつ？


### 問題2

次のことをやろう：

(1) クラスタ間距離のしきい値をいろいろ変えて結果を観察しよう．

(2) クラスタ間距離のしきい値が 6000 のときのクラスタ数と，各クラスタに属する画像の数の最大と最小をメモしよう．

(3) クラスタ数が 13 になるようなクラスタ間距離のしきい値を求め，そのときに各クラスタに属する画像の数の最大と最小をメモしよう．

----
## 演習問題 - $K$-平均法やってみよう
----

$K$-平均法の簡単な実験をやってみましょう．

In [ ]:
# 実験用データの入手
df = pd.read_csv('https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/data4kmeans.csv')
X = df[['x1', 'x2']].to_numpy()
print(X[:5, :], X.shape)

In [ ]:
# 散布図
fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(X[:, 0], X[:, 1])
ax.set_xlim(-5, 5)
ax.set_ylim(-5, 5)
ax.set_aspect('equal')
plt.show()

$K$-平均法では，データをいくつのクラスタに分けるかをあらかじめ指定する必要があります．また，クラスタリングの過程で，各クラスタに所属するデータの平均（これをセントロイドといいます）を計算します．クラスタ数を $K$ とすると，$K$ 個の平均(mean)を求めることになるので，K-means 法と呼ばれます．

ここでは，scikit-learn の [sklearn.cluster.KMeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) クラスを使い，上記の2次元データを $K$-平均法でクラスタリングする実験をやってみましょう．

次のセルの `K` にクラスタ数を指定して実行すると，配列 `X` に格納されたデータのクラスタリングが実行されます．配列 `label` にクラスタリング結果（各データが所属するクラスタの番号）が格納されます．


In [ ]:
K = 2 # クラスタ数

km = KMeans(n_clusters=K, verbose=0)
km.fit(X) # 学習
centroid = km.cluster_centers_ # セントロイド
label = km.predict(X) # 各データの所属クラスタを求める
print(label)
print()
print(f'K = {K}')
for ik in range(K):
    print(f'cluster{ik}: num = {np.sum(label == ik)}')

次のセルを実行すると，クラスタリングの結果を表示します．
各データ点および平面上の各点をクラスタリングして，それらの所属するクラスタによって異なる色を付けて表示します．また，セントロイドを ★ で表示します．


In [ ]:
# クラスタリング結果描画用のデータ
xmin, xmax = -5, 5
ymin, ymax = -5, 5
p = np.dstack(np.mgrid[xmin:xmax:0.05, ymin:ymax:0.05])
P = p.reshape((-1, p.shape[2]))
labelP = km.predict(P)

# クラスタ割り振り結果を描画
colors = seaborn.color_palette(n_colors=K)
fig = plt.figure(figsize=(13, 6))
ax0 = fig.add_subplot(121)
ax0.set_xlim(xmin, xmax)
ax0.set_ylim(ymin, ymax)
ax0.set_aspect('equal')
for ik in range(K):
    Xk = X[label==ik, :]
    Pk = P[labelP==ik, :]
    ax0.scatter(Xk[:, 0], Xk[:, 1], color=colors[ik])
    ax0.plot(centroid[ik, 0], centroid[ik, 1], color='white', marker='*', markerfacecolor=colors[ik], markersize=25)
    ax0.scatter(Pk[:, 0], Pk[:, 1], marker='.', alpha=0.1, color=colors[ik])

plt.show()

`K` の値をいろいろ変えて結果を観察してみましょう．ちなみに，$K$-平均法のアルゴリズムにはランダムな処理が含まれますので，同じデータで同じ`K`の設定でも，実行のたびに結果が変わります．